In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
os.chdir("..")
print(os.getcwd())
sys.path.insert(0, os.getcwd())

In [ ]:
import trimesh
import matplotlib
import numpy as np
import meshplot as mp
import matplotlib.pyplot as plt

# Visualize the Implicit Field

In [ ]:
# !ls -d pretrained/neural_fields/*/
!ls pretrained/neural_fields/jolteon/*

In [ ]:
log_path = "pretrained/neural_fields/jolteon/"

In [ ]:
import os.path as osp
from utils import load_imf

net, cfg = load_imf(
    log_path, 
    config_fpath=osp.join(log_path, "config.yaml"),
    ckpt_fpath=osp.join(log_path, "latest.pt"),
    return_cfg=True)
net, cfg

# Get the output Mesh

In [ ]:
from trainers.utils.vis_utils import imf2mesh
res = 256
out_mesh = imf2mesh(net, res=res, normalize=True, bound=1.2)

In [ ]:
npts = 1024*8
mesh_uniform = out_mesh.sample(npts)
mesh_uniform2 = out_mesh.sample(npts)

In [ ]:
p = mp.plot(out_mesh.vertices.astype(np.float32), out_mesh.faces.astype(np.float32)) 
p.add_points(mesh_uniform, shading={'point_size': 0.05})

# Sample using our Schema

In [ ]:
from trainers.utils.igp_utils import sample_points

nf_rejection, _ = sample_points(
    npts, dim=3, sample_surf_points=True, 
    out_nf=net, inp_nf=None, deform=None, 
    invert_sampling=False, detach_weight=True, use_rejection=True)
nf_rejection = nf_rejection.detach().cpu().numpy().reshape(-1, 3)
nf_rejection.shape

In [ ]:
p = mp.plot(out_mesh.vertices.astype(np.float32), out_mesh.faces.astype(np.float32)) 
p.add_points(nf_rejection, shading={'point_size': 0.05, 'point_color': 'blue'})

# Metrics

In [ ]:
import torch
from evaluation.evaluation_metrics import CD, EMD
x = torch.from_numpy(mesh_uniform).cuda().float().unsqueeze(0)
y = torch.from_numpy(mesh_uniform2).cuda().float().unsqueeze(0)
cd_base, dist_base = CD(x, y, return_dist=True, dist=None)
emd_base, _ = EMD(x, y, dist=dist_base)
cd_base.item(), emd_base.item()

In [ ]:
x = torch.from_numpy(mesh_uniform).cuda().float().unsqueeze(0)
y = torch.from_numpy(nf_rejection).cuda().float().unsqueeze(0)
cd_rej, dist_rej = CD(x, y, return_dist=True, dist=None)
emd_rej, _ = EMD(x, y, dist=dist_rej)
cd_rej.item(), emd_rej.item()

In [ ]:
"CDr", (cd_rej / cd_base * 100).item(), "EMDr", (emd_rej / emd_base * 100).item()